In [ ]:
pip install pydantic

In [ ]:
query = input("Enter your search query: ")

In [ ]:
limit = int(input("Enter total records to fetch: "))

In [ ]:
from app.ingest.primo_explore_client import explore_search
from app.ingest.primo_schema import brief_from_doc, PrimoFull
from app.ingest.primo_store import append_records

page_size = 10
results = []
fetched, offset = 0, 0

while fetched < limit:
    batch = min(page_size, limit - fetched)
    resp = explore_search(query=query, limit=batch, offset=offset)
    docs = resp.get("docs", [])
    if not docs:
        break
    for d in docs:
        results.append(brief_from_doc(d))
    fetched += len(docs)
    offset += len(docs)

# Show results before appending
for r in results:
    print(f"Title: {r.title}")
    print(f"Creators: {', '.join(r.creators) if r.creators else '—'}")
    print(f"Date: {r.creation_date or '—'}")
    print(f"Type: {r.resource_type or '—'}")
    print("-" * 40)

payload = [PrimoFull(brief=b, pnx={}, links={}, raw={}) for b in results]
wrote = append_records(payload, dedupe=True)
print(f"Exported {wrote} new records to /data/primo/records.jsonl.")